In [3]:
from MesoPy import Meso
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd

In [4]:
m = Meso(token='hidden')
precip = m.precip(stid='kfnl', start='201504241800', end='201504271200', units='precip|in')

In [5]:
m.timeseries('201912201942', '202001312130', stid = '1875P', vars = 'snow_depth')

In [6]:
from glob import glob
import rasterio as rio

In [7]:
# glob('/scratch/zacharykeskinen/jpl/*/tiffs/*')
fp = '/scratch/zacharykeskinen/jpl/lowman_23205_21019-018_21021-006_0006d_s01_L090_01_int_grd/tiffs/lowman_23205_21019-018_21021-006_0006d_s01_L090HH_01.cor.grd.tiff'
with rio.open(fp) as src:
    bounds  = src.bounds
    

RasterioIOError: /scratch/zacharykeskinen/jpl/lowman_23205_21019-018_21021-006_0006d_s01_L090_01_int_grd/tiffs/lowman_23205_21019-018_21021-006_0006d_s01_L090HH_01.cor.grd.tiff: No such file or directory

In [ ]:
bounds

bbox: list, optional
    Stations within a [lon/lat] box in the order [lonmin,latmin,lonmax,latmax] e.g. bbox="-120,40,-119,41"

In [ ]:
for stat in m.metadata(start = '202002010000', end = '202002100000', bbox = bounds)['STATION']:
    name = stat['NAME']
    long = stat['LONGITUDE']
    lat = stat['LATITUDE']
    elev = stat['ELEVATION']
    stid = stat['STID']
    status = stat['STATUS']
    print(name)
    print(lat)
    print(long)
    print(stid)

In [ ]:
def raster_box_extract(img, x_coord, y_coord, box_side = 5):
    meta = img.meta

    # Use the transform in the metadata and your coordinates
    try:
        rowcol = rio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)
        w = img.read(1, window=Window(rowcol[1], rowcol[0], box_side, box_side))
        return w
    except:
        pass
        # print('station outside bbox')

In [ ]:
stat_ls = []
stid_ls = []
res = {}
for stat in m.metadata(start = start_flight, end = end_flight, bbox = bounds)['STATION']:
    long = float(stat['LONGITUDE'])
    lat = float(stat['LATITUDE'])
    with rio.open(fp) as src:
        w = raster_box_extract(src, long, lat)
    if len(w[~np.isnan(w)]) > 0:
        if stat['NAME'].lower().replace(' ','') not in stat_ls:
            obs = m.timeseries(start_flight, end_flight, stid = stat['STID'], vars = 'snow_depth')
            if obs:
                obs = obs['STATION'][0]['OBSERVATIONS']
                d = {}
                dt = pd.to_datetime(obs['date_time'])
                if 'snow_depth_set_1' in obs.keys():
                    d['datetime'] = dt
                    d['img_arr'] = w
                    d['snow_depth_set_1'] = obs['snow_depth_set_1']
                    for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
                        if anc_col in obs.keys():
                            d[anc_col] = obs[anc_col]
                if d:
                    d['elev'] = stat['ELEVATION']
                    d['lat'] = stat['LATITUDE']
                    d['long'] = stat['LONGITUDE']
                    d['tz'] = stat['TIMEZONE']
                    res[stat['NAME']] = d
            
            stat_ls.append(stat['NAME'].lower().replace(' ',''))

In [ ]:
plt.imshow(res['Treeline']['img_arr'])

In [ ]:
stat_ls = []
stid_ls = []
for stat in m.metadata(start = start_flight, end = end_flight, bbox = bounds)['STATION']:
    long = float(stat['LONGITUDE'])
    lat = float(stat['LATITUDE'])
    with rio.open(fp) as src:
        w = raster_box_extract(src, long, lat)
    if len(w[~np.isnan(w)]) > 0:
        if stat['NAME'].lower().replace(' ','') not in stat_ls:

            stat = m.timeseries(start_flight, end_flight, stid = stat['STID'])
            res = {}
            d = {}
            obs = stat['OBSERVATIONS']

            dt = pd.to_datetime(obs['date_time'])
            if 'snow_depth_set_1' in obs.keys():
                d['datetime'] = dt
                d['snow_depth_set_1'] = obs['snow_depth_set_1']
                for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
                    if anc_col in obs.keys():
                        d[anc_col] = obs[anc_col]
            if d:
                d['elev'] = stat['ELEVATION']
                d['lat'] = stat['LATITUDE']
                d['long'] = stat['LONGITUDE']
                d['tz'] = stat['TIMEZONE']
                res[stat['NAME']] = d
        stat_ls.append(stat['NAME'].lower().replace(' ',''))

In [ ]:
a = m.timeseries(start = start_flight, end = end_flight, stid = stid_ls)

res = {}
for stat in a['STATION']:
    # print(stat['SENSOR_VARIABLES'])
    d = {}
    obs = stat['OBSERVATIONS']

    dt = pd.to_datetime(obs['date_time'])
    if 'snow_depth_set_1' in obs.keys():
        d['datetime'] = dt
        d['snow_depth_set_1'] = obs['snow_depth_set_1']
        for anc_col in ['air_temp_set_1','snow_water_equiv_set_1']:
            if anc_col in obs.keys():
                d[anc_col] = obs[anc_col]
    if d:
        d['elev'] = stat['ELEVATION']
        d['lat'] = stat['LATITUDE']
        d['long'] = stat['LONGITUDE']
        d['tz'] = stat['TIMEZONE']
        res[stat['NAME']] = d

In [ ]:
for col in ann_df.columns:
    if 'acq' in col:
        print(col)

In [ ]:
ann_df = pd.read_csv('/home/zacharykeskinen/uavsar/data/slc_stack/lowman_23205_20002-007_20007-003_0013d_s01_L090_01_int_grd/lowman_23205_20002-007_20007-003_0013d_s01_L090_01_int_grd.csv')
start = pd.to_datetime(ann_df['stop time of acquisition for pass 1'][0])
end = pd.to_datetime(ann_df['start time of acquisition for pass 2'][0])

In [ ]:
def mesopy_date_parse(pd_date_str):
    return pd_date_str.strftime('%Y') + pd_date_str.strftime('%m') + pd_date_str.strftime('%d') + pd_date_str.strftime('%H') + pd_date_str.strftime('%M')

In [ ]:
mesopy_date_parse(pd.to_datetime('2020-10-04T20:30'))

In [ ]:
import datetime
isinstance((pd.to_datetime('2020-10-01')), datetime.datetime)


In [ ]:
start.month

In [ ]:
start_flight = '202001010000'
end_flight = '202001070000'
a = m.timeseries(start_flight, end_flight, stid = 'BASI1' )
dt = a['STATION'][0]['OBSERVATIONS']['date_time']
sd = a['STATION'][0]['OBSERVATIONS']['snow_depth_set_1']
swe = a['STATION'][0]['OBSERVATIONS']['snow_water_equiv_set_1']
temp = a['STATION'][0]['OBSERVATIONS']['air_temp_set_1']
a['STATION'][0]['OBSERVATIONS'].keys()

In [ ]:
from rasterio.windows import Window    
with rio.open(fp) as src:
    meta = src.meta

    # My target coordinates
    x_coord = -115.23447
    y_coord = 44.30342

    # Use the transform in the metadata and your coordinates
    rowcol = rio.transform.rowcol(meta['transform'], xs=x_coord, ys=y_coord)

    w = src.read(1, window=Window(rowcol[1], rowcol[0], 5, 5))
    # Do something with your data here.
    print(w)

In [ ]:
df =pd.read_csv('/home/zacharykeskinen/uavsar/results/test_wx.csv')

In [ ]:
plt.plot(df.set_index('Unnamed: 0')['BANNER SUMMIT']['snow_depth_set_1'])

In [ ]:
import pickle
with open('/home/zacharykeskinen/uavsar/results/test_wx.pkl', 'rb') as f:
    res  = pickle.load(f)

In [ ]:
plt.scatter(res['Lower Weather']['snow_depth_set_1'])

In [ ]:
for site,obs in res.items():
    print(site)
    #print(obs)
    plt.scatter(obs['datetime'], obs['snow_depth_set_1'])
    plt.show()